# Lab 2: Recommender systems

Following the second lecture of this week, we will now zoom in a bit more into recommender systems.

More specifically, we are going to try out matrix factorization on a slice of the first Movielens dataset.

While doing this, you also will be introduced to a few handy data handling routines in Python.

## About the Movielens data
Movielens (https://movielens.org/) is a movie recommendation service run by the Grouplens (http://grouplens.org/) team at the University of Minnesota. Over the years, the service has been growing considerably, and once every few years, Grouplens releases new anonymized usage data to the community (http://grouplens.org/datasets/movielens/), which have become de facto benchmarking standards in the Recommender Systems research community.

In 1998, the first released dataset consisted of 100,000 ratings (1000 users, 1700 movies). The most recently released benchmark dataset (2014) consisted of 20 million ratings (138,000 users, 27,000 movies).

In the current lab session, we will not focus on the big data crunching, but rather on getting a better practical feeling on how to handle user-item data, as well as the implications of Matrix Factorization.

## What to hand in
As final deliverable to demonstrate your successful completion of this assignment, please go to https://www.dropbox.com/request/UR4fXWKPsDRdjKNKhVZY and submit a file named [studentNumberMember1_studentNumberMember2.pdf].

This file should:
* Include **three** latent factor scatter plots, obtained for **three different input data slices** (which should differ at least in terms of user population size).
* For each of the scatter plots, please:
  * specify the characteristics of your input data (e.g. population size, selection criteria for users and films);
  * based on the plot, give a possible interpretation of what the factors represented by the x and y axes of your plot may indicate.

Further instructions can be found further down this notebook, at the point where we give an example plot.

## Importing the data
We have prepared a subset of the Movielens 100k dataset, which should automagically be downloaded to your (virtual) machine:

In [1]:
from datasets import CS4065_Dataset
movielens_paths = CS4065_Dataset.get_movielens_subset()

<code>movielens_paths</code> is a dictionary, keyed by three file names in the Movielens 100k dataset:
* <code>u.data</code>, containing tab-separated rating data (specifying user ID, movie ID, rating, timestamp);
* <code>u.user</code>, containing pipe-separated anonymized demographics data for all users in the dataset (specifying user ID, age, gender, occupation, ZIP code);
* <code>u.item</code>, containing pipe-separated movie background information (specifying movie ID, title, genres, IMDB URLs, and more).

In [2]:
# Verify the contents of movielens_paths
print movielens_paths

{'u.item': '/Users/Shashank/Workspace_Dev/TUDelft/MultiMedia/cs4065/data/movielens_subset/u.item', 'u.user': '/Users/Shashank/Workspace_Dev/TUDelft/MultiMedia/cs4065/data/movielens_subset/u.user', 'u.data': '/Users/Shashank/Workspace_Dev/TUDelft/MultiMedia/cs4065/data/movielens_subset/u.data'}


## Selecting analysis data

In this lab, we will primarily use data from <code>u.item</code>. The information in <code>u.user</code> is interesting though for assessing potential characterizing user features, and the information in <code>u.item</code> was used by us to manually select a small set of movie IDs, for which we expect that the fan base will show some variation.

We put our selection in a <code>movie_data</code> dictionary:

In [3]:
# Initialization of the dictionary.
# It will be keyed by movie ID, and have the IMDB movie title as corresponding value.
movie_data = {}

# We now populate the dictionary with some manually chosen examples,
# for which we expect to see some different underlying fan bases.

# First, we add three animated movies about princesses.
movie_data[418] = 'Cinderella (1950)'
movie_data[538] = 'Anastasia (1997)'
movie_data[542] = 'Pocahontas (1995)'
# Then, we add three psychological horror movies.
movie_data[200] = 'Shining, The (1980)'
movie_data[98] = 'Silence of the Lambs, The (1991)'
movie_data[185] = 'Psycho (1960)'
# Subsequently, we add three musical movies.
movie_data[186] = 'Blues Brothers, The (1980)'
movie_data[451] = 'Grease (1978)'
movie_data[289] = 'Evita (1996)'
# And finally, we add three movies taking place in space.
movie_data[50] = 'Star Wars (1977)'
movie_data[89] = 'Blade Runner (1982)'
movie_data[135] = '2001: A Space Odyssey (1968)'

# movie_data[127] = 'Godfather, The (1972)'
# movie_data[423] =  'E.T. the Extra-Terrestrial (1982)'
# movie_data[485] = 'My Fair Lady (1964)'

# Now, we have a dictionary with 12 items:
print movie_data
print '\nMovie %d is titled "%s".' % (200, movie_data[200])

{289: 'Evita (1996)', 418: 'Cinderella (1950)', 451: 'Grease (1978)', 135: '2001: A Space Odyssey (1968)', 200: 'Shining, The (1980)', 98: 'Silence of the Lambs, The (1991)', 50: 'Star Wars (1977)', 89: 'Blade Runner (1982)', 185: 'Psycho (1960)', 538: 'Anastasia (1997)', 186: 'Blues Brothers, The (1980)', 542: 'Pocahontas (1995)'}

Movie 200 is titled "Shining, The (1980)".


## Extracting the rating data

Next step: we won't look at 100,000 ratings, but **only at ratings for any of these movies**.

Let's first put the IDs of the movies we are interested in in a separate variable.

In [4]:
movies_considered = movie_data.keys()
# For convenience, we will sort the movies by ID.
movies_considered = sorted(movies_considered)
# How many movies will we consider again?
len(movies_considered)
# movies_considered

12

For extracting the rating data, we will use the <code>pandas</code> module. It offers some more advanced data structures and corresponding handling capabilities than simple Python dictionaries would be capable of.

We will illustrate some of the possibilities here, but see http://pandas.pydata.org/ and the documentation at http://pandas.pydata.org/pandas-docs/version/0.17.1/ for more extensive background and more advanced examples.

In [5]:
import pandas

A first neat feature is that we can use the <code>read_csv</code> function to read formatted data into a so-called DataFrame:

In [6]:
# let's check the help documentation for read_csv
help(pandas.read_csv)

Help on function read_csv in module pandas.io.parsers:

read_csv(filepath_or_buffer, sep=',', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, iterator=False, chunksize=None, compression='infer', thousands=None, decimal='.', lineterminator=None, quotechar='"', quoting=0, escapechar=None, comment=None, encoding=None, dialect=None, tupleize_cols=False, error_bad_lines=True, warn_bad_lines=True, skipfooter=0, skip_footer=0, doublequote=True, delim_whitespace=False, as_recarray=False, compact_ints=False, use_unsigned=False, low_memory=True, buffer_lines=None, memory_map=False, float_precision=Non

Clearly, there are many options.
What we want to do in the current case, is reading in the <code>u.data</code> file, and giving it proper headers:

In [7]:
all_rating_data = pandas.read_csv(
    movielens_paths['u.data'], # path to file to be read
    sep = '\t', # the data we are reading is tab-separated
    names = ['user_id', 'movie_id', 'rating', 'timestamp'] # custom indicated names for the columns to be read
)

In the way they are printed, dataframes may somewhat remind you of database tables:

In [8]:
all_rating_data.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


Some further possibilities:

In [9]:
# return the data at row 3
print all_rating_data.loc[3]

user_id            244
movie_id            51
rating               2
timestamp    880606923
Name: 3, dtype: int64


In [10]:
# return the rating of the data at row 3
print all_rating_data.loc[3]['rating']

2


In [11]:
# find all rows corresponding to ratings for Cinderella (movie ID 418)
cinderella_ratings = all_rating_data.loc[all_rating_data['movie_id'] == 418]
cinderella_ratings

,user_id,movie_id,rating,timestamp
984,13,418,2,882398763
1010,275,418,3,875154718
1074,291,418,4,875086920
1198,303,418,4,879483510
1408,59,418,2,888205188
2384,254,418,3,886473078
2702,128,418,4,879968164
4732,130,418,5,875801631
5092,298,418,4,884183406
5407,290,418,3,880474293


## Question
How many people rated Cinderella?

In [12]:
# enter your code here
len(cinderella_ratings['user_id'])

129

## Question
Using the <code>cinderella_ratings</code> variable, extract and print the ratings of people who hated Cinderella (that is, who gave the movie a rating of 1).

In [13]:
# replace 'None' below by your code to do the proper extraction
cinderella_hate_ratings = None
cinderella_hate_ratings = cinderella_ratings.loc[cinderella_ratings['rating'] == 1]
# add your printing statement here 
cinderella_hate_ratings

,user_id,movie_id,rating,timestamp
58689,608,418,1,880405971
90549,524,418,1,884637598
95254,865,418,1,880235099


We might as well check the demographics characteristics of the people who gave Cinderella such a low score.

Demographics info is available in the file at location <code>movielens_paths['u.user']</code>.
As we stated before, this info is pipe-separated, a sample line is:

<code>14|45|M|scientist|55106</code>.

The values indicated correspond to the elements

<code>user_ID|age|gender|occupation|ZIP_code</code>.

## Question
Read the user demographics data into a variable <code>user_data</code>.

In [14]:
user_data = None # replace None by your code
user_data = pandas.read_csv(movielens_paths['u.user'], # path to file to be read
    sep = '|', # the data we are reading is tab-separated
    names = ['user_id', 'age', 'gender', 'occupation', 'zipcode'] # custom indicated names for the columns to be read
)

# Uncomment the following line to display the demographics information of the Cinderella haters.
user_data.loc[user_data['user_id'].isin(cinderella_hate_ratings['user_id'])]

,user_id,age,gender,occupation,zipcode
523,524,56,M,educator,02159
607,608,22,M,other,10003
864,865,25,M,artist,11231


## Selecting data of interest
We only want to keep ratings for the movies we are interested in. For this, we can make use of membership testing on our <code>movies_considered</code> variable:

In [15]:
ratings_for_movies = all_rating_data.loc[all_rating_data['movie_id'].isin(movies_considered)]

In [16]:
# how many ratings do we have for our considered movies?
# ratings_for_movies
len(ratings_for_movies)

2878

pandas offers <code>unique()</code> and <code>nunique()</code> functions which are similar to <code>DISTINCT</code> and <code>COUNT(DISTINCT)</code> in SQL:

In [17]:
# how many unique users gave the ratings for our movies of interest?
print ratings_for_movies['user_id'].nunique()
print ratings_for_movies['movie_id'].nunique()

775
12


## Question
What is the sparsity proportion of the user-item matrix of the <code>ratings_for_movies</code> data?

That is, what is the ratio of zero-valued (so unrated) items over the total number of elements in the user-item matrix?

In [39]:
# replace the code below by your code to compute the sparsity proportion
1 - (2878.0/(775*12))

0.6905376344086022

## Starting small: focusing on actively rating users
As a first small example, let's first focus on a small set of users.

Let's say we want to select the 10 users who rated most of the movies of our interest.

Using <code>groupby</code> and <code>unique()</code>, we can flatten our rating table to collect a single list of rated movies per unique user ID:

In [19]:
users_to_rated_movies = ratings_for_movies.groupby('user_id')['movie_id'].unique()
# which movie IDs were rated by user 1?
users_to_rated_movies[1]

array([ 98, 186, 185, 200, 135,  89,  50])

But we can also directly call <code>nunique()</code> to find the number of rated movies per user:

In [20]:
users_to_rating_count = ratings_for_movies.groupby('user_id')['movie_id'].nunique()
# how many movies were rated by user 1?
users_to_rating_count[1]

7

To identify the most active users, we sort the user IDs based on the amount of movies they rated, in descending order.

In [21]:
users_sorted_by_rating_count = users_to_rating_count.sort_values(ascending = False)
print users_sorted_by_rating_count.head()

user_id
13     11
378    10
70     10
234    10
222    10
Name: movie_id, dtype: int64


We now select the 10, 15, and 500 most active users.

In [103]:
users_considered_10 = users_sorted_by_rating_count[0:10].keys()
users_considered_15 = users_sorted_by_rating_count[0:15].keys()
users_considered_500 = users_sorted_by_rating_count[0:500].keys()
# again, for convenience we sort our table
# users_considered = sorted(users_considered)
users_considered_10 = sorted(users_considered_10)
users_considered_15 = sorted(users_considered_15)
users_considered_500 = sorted(users_considered_500)

# is user 1 among the most active users?
print 1 in users_considered_15
# is user 13 among the most active users?
print 13 in users_considered_15

False
True


In [104]:
# If you want to see the demographics of the selected users, uncomment the following line.
user_data.loc[user_data['user_id'].isin(users_considered_15)]

,user_id,age,gender,occupation,zipcode
6,7,57,M,administrator,91344
12,13,47,M,educator,29206
69,70,27,M,engineer,60067
94,95,31,M,administrator,10707
193,194,38,M,administrator,02154
221,222,29,M,programmer,27502
233,234,60,M,retired,94702
275,276,21,M,student,95064
312,313,41,M,marketing,60035
377,378,35,M,student,02859


## Preparing the matrix factorization
We now know what movies we are interested in, and what users we are interested in. Now all we need to do is establishing a user-item matrix, and applying factorization on it.

We use numpy for handling matrices.

In [105]:
import numpy as np

## Question
What are the dimensions of the user-item matrix, which we will call $\mathbf{R}$ (as it holds ratings)?

Enter the appropriate number of rows and columns below, so we can initialize an all-zero matrix with the proper size.

In [106]:
num_rows = 15 # Replace 1000 by the desired number of rows
num_columns = 12 # Replace 1000 by the desired number of columns

R = np.zeros((num_rows, num_columns))

There is one more thing to resolve. Our user IDs and movie IDs are numbers, but they do not reflect the desired coordinates in a user-item matrix.

That is, R[0,0] should reflect the rating of 'the first user' on 'the first movie'.

In [107]:
# What is the lowest user ID we have?
print users_considered_15[0]
# What is the lowest movie ID we have?
print movies_considered[0]
print 'So, R[0,0] should reflect the rating of user %d on movie %d.' % (users_considered_15[0], movies_considered[0])

7
50
So, R[0,0] should reflect the rating of user 7 on movie 50.


We therefore will create lookup maps, which map user and movie IDs to a matrix row/column index [0, 1, 2...], and the other way around.

In [108]:
user_to_index = dict(zip(users_considered_15, range(len(users_considered_15))))
movie_to_index = dict(zip(movies_considered, range(len(movies_considered))))
index_to_user = dict(zip(user_to_index.values(), user_to_index.keys()))      
index_to_movie = dict(zip(movie_to_index.values(), movie_to_index.keys()))

print user_to_index
print movie_to_index
print index_to_user
print index_to_movie

# feel free to print them to verify that the mapping worked out correctly.
# Note that the keys() of a dictionary are not necessarily returned in order.

{194: 4, 774: 12, 70: 2, 7: 0, 234: 6, 13: 1, 846: 13, 881: 14, 276: 7, 405: 10, 406: 11, 313: 8, 378: 9, 222: 5, 95: 3}
{289: 7, 98: 2, 451: 9, 135: 3, 200: 6, 418: 8, 50: 0, 185: 4, 89: 1, 186: 5, 538: 10, 542: 11}
{0: 7, 1: 13, 2: 70, 3: 95, 4: 194, 5: 222, 6: 234, 7: 276, 8: 313, 9: 378, 10: 405, 11: 406, 12: 774, 13: 846, 14: 881}
{0: 50, 1: 89, 2: 98, 3: 135, 4: 185, 5: 186, 6: 200, 7: 289, 8: 418, 9: 451, 10: 538, 11: 542}


At last, we now will populate the user-item matrix.

We do this by iterating over the relevant data frame rows, and then extracting the movie ID and rating score in case we encounter a user of interest.

In [109]:
ratings_for_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2878 entries, 8 to 99991
Data columns (total 4 columns):
user_id      2878 non-null int64
movie_id     2878 non-null int64
rating       2878 non-null int64
timestamp    2878 non-null int64
dtypes: int64(4)
memory usage: 112.4 KB


In [110]:
for index, row in ratings_for_movies.iterrows():
    if row['user_id'] in users_considered_15:
        i = user_to_index[row['user_id']]
        j = movie_to_index[row['movie_id']]
        R[i, j] = row['rating']


In [111]:
# What is the user-item matrix looking like?
print R

[[ 5.  5.  4.  5.  5.  4.  5.  0.  4.  5.  0.  4.]
 [ 5.  4.  4.  5.  3.  4.  3.  2.  2.  1.  1.  0.]
 [ 4.  4.  0.  4.  4.  4.  0.  3.  3.  4.  2.  2.]
 [ 5.  3.  4.  3.  3.  5.  2.  2.  0.  3.  0.  2.]
 [ 3.  3.  4.  3.  4.  5.  0.  1.  0.  2.  0.  3.]
 [ 4.  5.  4.  5.  4.  5.  3.  0.  2.  3.  0.  2.]
 [ 4.  3.  4.  4.  3.  3.  5.  4.  3.  3.  0.  0.]
 [ 5.  5.  5.  5.  4.  5.  5.  2.  4.  3.  0.  0.]
 [ 5.  5.  4.  5.  5.  3.  3.  0.  3.  0.  2.  3.]
 [ 4.  4.  5.  2.  0.  3.  3.  5.  3.  4.  0.  4.]
 [ 5.  1.  4.  5.  4.  5.  2.  0.  5.  5.  0.  1.]
 [ 5.  4.  4.  5.  5.  3.  0.  3.  5.  2.  0.  0.]
 [ 4.  2.  4.  3.  2.  3.  2.  0.  2.  1.  0.  0.]
 [ 5.  5.  4.  4.  5.  5.  4.  4.  0.  4.  0.  3.]
 [ 3.  4.  5.  4.  5.  3.  2.  1.  0.  1.  0.  1.]]


For applying the matrix factorization, we use the <code>svd()</code> function in <code>scipy.linalg</code> here.

*Note that there are a few Singular Value Decomposition implementations offered by scipy and scikits-learn, also with dedicated approaches targeted at sparse matrices. But for our current smaller samples, the regular SVD works fine.*

In [112]:
import scipy.linalg

In [113]:
# How to call the SVD function?
help(scipy.linalg.svd)

Help on function svd in module scipy.linalg.decomp_svd:

svd(a, full_matrices=True, compute_uv=True, overwrite_a=False, check_finite=True, lapack_driver='gesdd')
    Singular Value Decomposition.
    
    Factorizes the matrix a into two unitary matrices U and Vh, and
    a 1-D array s of singular values (real, non-negative) such that
    ``a == U*S*Vh``, where S is a suitably shaped matrix of zeros with
    main diagonal s.
    
    Parameters
    ----------
    a : (M, N) array_like
        Matrix to decompose.
    full_matrices : bool, optional
        If True, `U` and `Vh` are of shape ``(M,M)``, ``(N,N)``.
        If False, the shapes are ``(M,K)`` and ``(K,N)``, where
        ``K = min(M,N)``.
    compute_uv : bool, optional
        Whether to compute also `U` and `Vh` in addition to `s`.
        Default is True.
    overwrite_a : bool, optional
        Whether to overwrite `a`; may improve performance.
        Default is False.
    check_finite : bool, optional
        Whether t

We now perform the SVD. Then, we can express $\mathbf{R}$ as

$$\mathbf{R} = \mathbf{U} \mathbf{\Sigma} \mathbf{V^T}$$

with $\mathbf{U}$ being an orthonormal matrix expressing user-related factors,
$\mathbf{\Sigma}$ being a diagonal matrix expressing singular values of decreasing weight,
and $\mathbf{V}$ being an orthonormal matrix expressing movie-related factors,


In [114]:
U, S, Vt = scipy.linalg.svd(R, full_matrices=False)

In [115]:
print U.shape
print S.shape
print Vt.shape

(15, 12)
(12,)
(12, 12)


In [116]:
# What are the first 10 singular values?
print S[0:10]
len(S)

[ 43.33711919   7.80509779   7.24957965   6.60164182   5.84450433
   5.13350515   4.31204747   3.1468138    2.30062918   1.42144877]


12

## Plotting movies in latent factor space
Based on our analysis, we will plot the movies in latent factor space to see to what extent they cluster or contrast.

For the plotting, we will use matplotlib:

In [36]:
import matplotlib.pyplot as plt
%matplotlib notebook
# add the 'magic' function above to ensure plots are rendered within the notebook
# for this, you either can use %matplot inline (which we did in the past lab), which will plot inline
# or % matplotlib notebook (which we do here), which will give some additional plot interaction possibilities

## Make 2D scatterplots of factor weights
We select the indices corresponding to the factors we want to examine.

Let's first take the first two factors, as they have the strongest weight.

In [122]:
factor_x_index = 1
factor_y_index = 3

<IPython.core.display.Javascript object>


In [123]:
plt.figure();

Then, we can use the <code>scatter</code> function to draw a scatter plot.

In [124]:
# Recall that Vt is holding the movie-related factors

plt.scatter(Vt[factor_x_index,:], Vt[factor_y_index,:])
for movie_index in range(len(movies_considered)):
    plt.annotate(
        movie_data[index_to_movie[movie_index]],
        (Vt[factor_x_index, movie_index], Vt[factor_y_index, movie_index]))
plt.xlabel('factor %d' % (factor_x_index + 1))
plt.ylabel('factor %d' % (factor_y_index + 1))
# optional: save higher-resolution *.png locally
# plt.savefig('name_of_plot', ext='png', dpi=150)
plt.savefig('15fac13usertest', ext='png', dpi=150)
plt.show()

## Rating prediction using matrix factorization
We can use matrix factorization to predict user ratings.

Looking at our 10 users, let's remove one from the user-item matrix, and try to predict the ratings for this user.

Say we remove the fourth user from <code>users_considered</code>:

In [125]:
user_to_predict = users_considered_15[3]
ratings_to_predict = R[3,:]
print 'to predict: %s for user %s.' % (ratings_to_predict, user_to_predict)

to predict: [ 5.  3.  4.  3.  3.  5.  2.  2.  0.  3.  0.  2.] for user 95.


We remove this user from R and re-establish the factorization on the new matrix.

In [126]:
# remove the user from users_considered
users_considered_15.remove(users_considered_15[3])
# redo the lookup indices
user_to_index = dict(zip(users_considered_15, range(len(users_considered_15))))
index_to_user = dict(zip(user_to_index.values(), user_to_index.keys()))      

# remove user from R
R = np.delete(R, (3), axis=0)
R

array([[ 5.,  5.,  4.,  5.,  5.,  4.,  5.,  0.,  4.,  5.,  0.,  4.],
       [ 5.,  4.,  4.,  5.,  3.,  4.,  3.,  2.,  2.,  1.,  1.,  0.],
       [ 4.,  4.,  0.,  4.,  4.,  4.,  0.,  3.,  3.,  4.,  2.,  2.],
       [ 3.,  3.,  4.,  3.,  4.,  5.,  0.,  1.,  0.,  2.,  0.,  3.],
       [ 4.,  5.,  4.,  5.,  4.,  5.,  3.,  0.,  2.,  3.,  0.,  2.],
       [ 4.,  3.,  4.,  4.,  3.,  3.,  5.,  4.,  3.,  3.,  0.,  0.],
       [ 5.,  5.,  5.,  5.,  4.,  5.,  5.,  2.,  4.,  3.,  0.,  0.],
       [ 5.,  5.,  4.,  5.,  5.,  3.,  3.,  0.,  3.,  0.,  2.,  3.],
       [ 4.,  4.,  5.,  2.,  0.,  3.,  3.,  5.,  3.,  4.,  0.,  4.],
       [ 5.,  1.,  4.,  5.,  4.,  5.,  2.,  0.,  5.,  5.,  0.,  1.],
       [ 5.,  4.,  4.,  5.,  5.,  3.,  0.,  3.,  5.,  2.,  0.,  0.],
       [ 4.,  2.,  4.,  3.,  2.,  3.,  2.,  0.,  2.,  1.,  0.,  0.],
       [ 5.,  5.,  4.,  4.,  5.,  5.,  4.,  4.,  0.,  4.,  0.,  3.],
       [ 3.,  4.,  5.,  4.,  5.,  3.,  2.,  1.,  0.,  1.,  0.,  1.]])

In [127]:
# redo the SVD
U, S, Vt = scipy.linalg.svd(R, full_matrices=False)

<IPython.core.display.Javascript object>


In [128]:
# make a new scatter plot
plt.figure()
plt.scatter(Vt[factor_x_index,:], Vt[factor_y_index,:])
for movie_index in range(len(movies_considered)):
    plt.annotate(
        movie_data[index_to_movie[movie_index]],
        (Vt[factor_x_index, movie_index], Vt[factor_y_index, movie_index]))
plt.xlabel('factor %d' % (factor_x_index + 1))
plt.ylabel('factor %d' % (factor_y_index + 1))
plt.savefig('rat15usertest', ext='png', dpi=150)
plt.show()

Note how this plot differs from your previous scatter plot.

The SVD solving approach suffers from sign indeterminacy, which means that the sign corresponding to a factor may flip for different initializations. Still, internal relationships should reasonably hold, and that indeed is the case (e.g. on factor 2, Psycho and Evita are still strong opposites).

## Performing rating prediction
Say we will only consider the first two factors, and try to predict the ratings using this model for our user 222.

We will first project the user's rating into 2D factor space by multiplying with $\mathbf{V}$:

In [129]:
Vt[0:2,:].transpose()

array([[-0.38939493,  0.04101673],
       [-0.34763946, -0.02231469],
       [-0.34800951, -0.01126312],
       [-0.37819248,  0.29767722],
       [-0.34066704,  0.44766951],
       [-0.34974716,  0.0537295 ],
       [-0.24674872, -0.24737148],
       [-0.15709718, -0.65541215],
       [-0.23443282,  0.01982673],
       [-0.24806227, -0.38454822],
       [-0.03024483,  0.08761479],
       [-0.14841922, -0.24306733]])

In [130]:
ratings_to_predict

array([ 5.,  3.,  4.,  3.,  3.,  5.,  2.,  2.,  0.,  3.,  0.,  2.])

In [131]:
ratings_to_predict_2d = np.dot(ratings_to_predict, Vt[0:2,:].transpose())

Then, reconstruct the data by multiplying with  $\mathbf{V^T}$:

In [132]:
ratings_to_predict_reconstructed = np.dot(ratings_to_predict_2d, Vt[0:2,:])
print ratings_to_predict_reconstructed
print ratings_to_predict

[ 3.91212777  3.54257387  3.5369577   3.58104197  3.07355628  3.49948448
  2.71070089  2.14784003  2.35939774  2.84028214  0.23230064  1.71038868]
[ 5.  3.  4.  3.  3.  5.  2.  2.  0.  3.  0.  2.]


To get a better feel for how accurate the prediction is, implement the RMSE measure:

In [133]:
def RMSE(array1, array2):
    # Replace -1 by a proper RMSE (Root Mean Square Error) implementation.
    # You will at least need np.sqrt() -- consult the numpy documentation and use the IPython tab completion to further
    # establish the necessary functionality.
    rmse_val = np.sqrt(np.sum(np.dot((array1-array2),(array1-array2).transpose())))
    rmse_val = rmse_val/len(array1)
    return rmse_val

# what is the RMSE for our example?
RMSE(ratings_to_predict, ratings_to_predict_reconstructed)

0.27059458612995529

## I don't see how this SVD reconstruction works?
If you need to read up on your SVD background, check http://infolab.stanford.edu/~ullman/mmds/ch11.pdf and the corresponding examples.

## Wrapping up this lab assignment
We now gave you examples of how to perform matrix factorization on a movie ratings dataset.

For your lab course deliverable of this week, please play around with these examples more, and examine at least the effect of using different sizes of user populations.

* Our current example only used 10 (or 9) users, what would happen if you use slightly more, or many more? How are the main latent factors affected, how is RMSE affected?

* In the scatter plot, play around with different factors. Instead of the first two, you can for example also test the first and the third.

* Optional: What would happen to the factorization if you would include other movies? Some nice ones to try are:
<code>
movieID 127: 'Godfather, The (1972)'
movieID 423: 'E.T. the Extra-Terrestrial (1982)'
movieID 485: 'My Fair Lady (1964)'
</code>
but feel free to consult <code>u.item</code> to select the IDs and titles of movies you find interesting.

As indicated above, hand in a \*.pdf report named<code>[studentNumberMember1_studentNumberMember2.pdf]</code> in which you include **three** latent factor scatter plots, obtained for **three different input data slices**, which at least differ in terms of user population size (so the size of <code>users_considered</code>.

For each of the scatter plots, do two things:
  * specify the characteristics of your input data (e.g. population size, selection criteria for users and films);
  * based on each plot, give a possible interpretation of what the factors represented by the x and y axes of your plot may indicate.

You can simply retrieve the scatter plot images for your report by using the 'download' button underneath each plot.

Alternatively, you can export higher-resolution images through <code>savefig</code>, e.g.

<code>$ plt.savefig('name_of_plot', ext='png', dpi=150)</code>.

Place this call before calling <code>plt.show()</code>; also see the commented line accompanying the first scatter plot.
